# Join DB

In this notebook we create the dataset we will use in the future. In particular we joined some of the .csv that we can find in Mimic so that we have all the informations about the hospitalizations and the clinical notes in a single dataset.

In [1]:
import pandas as pd
import importlib
import pickle
from nltk.tokenize import RegexpTokenizer
from os import listdir
from os.path import isfile, join
sys.path.append('..')
from lib.utils import Utils
import lib.NotesCleaning
from lib.NotesCleaning import NotesCleaning
importlib.reload(lib.NotesCleaning)
pd.set_option('display.max_columns', None)

## Notes dataset

First of all we load the csv with all the notes and then we clean it using the clean_note method.

In [2]:
notes_csv = pd.read_csv("../../Dati/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv", low_memory=False)

In [3]:
notes_csv.shape

(2083180, 11)

In [4]:
notes_csv['Cleaned_Text'] = notes_csv.TEXT.apply(lambda x: NotesCleaning().clean_note(x))

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
notes_csv['Token'] = notes_csv['Cleaned_Text'].apply(lambda x: NotesCleaning().tokenize(x, tokenizer))

## Admission and diagnoses dataset

Now we load the admission dataset and the diagnoses dataset.
These two were cleaned using the pre_processing function of Doctor XAI.

In [15]:
# Load the already cleaned csv
admission_df = pd.read_csv("../../Dati/mimic-iii-clinical-database-1.4/CSV/clean_ADMISSIONS.csv")
diagnoses_csv = pd.read_csv("../../Dati/mimic-iii-clinical-database-1.4/CSV/clean_DIAGNOSES_ICD.csv")
diagnoses_df = diagnoses_csv.groupby(['SUBJECT_ID','HADM_ID'], as_index=False).agg( {'ICD9_CODE':list })

## Join the datasets

In [8]:
tmp_join = admission_df.merge(diagnoses_df, on=['SUBJECT_ID', 'HADM_ID'])

In [9]:
df = notes_csv.merge(tmp_join, on=['SUBJECT_ID', 'HADM_ID'])

In [10]:
pd.set_option('display.max_columns', None)
df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,Cleaned_Text,Token,ROW_ID_y,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ICD9_CODE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,service addendum radiologic studies radiol...,"[service, addendum, radiologic, studies, radio...",27478,2151-07-16 14:29:00,2151-08-04 19:10:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."
1,170,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,history of present illness the patient is an...,"[history, present, illness, patient, year, old...",27478,2151-07-16 14:29:00,2151-08-04 19:10:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."
2,59795,22532,167853.0,2151-08-02,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,patient test information indication aortic v...,"[patient, test, information, indication, aorti...",27478,2151-07-16 14:29:00,2151-08-04 19:10:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."
3,59796,22532,167853.0,2151-07-19,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Endocar...,patient test information indication endocard...,"[patient, test, information, indication, endoc...",27478,2151-07-16 14:29:00,2151-08-04 19:10:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."
4,106289,22532,167853.0,2151-08-03,NaN,NaN,ECG,Report,NaN,NaN,Atrial fibrillation with a slow ventricular re...,atrial fibrillation with a slow ventricular re...,"[atrial, fibrillation, slow, ventricular, resp...",27478,2151-07-16 14:29:00,2151-08-04 19:10:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."


In [12]:
df.shape

(1851243, 31)

In [ ]:
cleanedText = pd.DataFrame()

In [ ]:
cleanedText['Cleaned_Text'] = df['Cleaned_Text']

In [34]:
cleanedText.to_csv('../data/cleanedText.csv')

In [13]:
df = df[['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'INSURANCE', 'ADMISSION_TYPE', 'INSURANCE', 'TEXT', 'Token', 'ICD9_CODE', 'DIAGNOSIS']]

In [14]:
df.to_csv("../data/merged_dataset.csv")

## Reduced dataset with only "Discharge summary" notes

In [41]:
df.shape

(1851243, 11)

In [39]:
df_discharge = df[df.CATEGORY == "Discharge summary"]

In [45]:
df_discharge.shape

(59646, 11)

In [47]:
df_discharge.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,CATEGORY,INSURANCE,ADMISSION_TYPE,INSURANCE.1,TEXT,Token,ICD9_CODE,DIAGNOSIS
0,0,22532,167853.0,Discharge summary,Medicare,EMERGENCY,Medicare,Admission Date: [**2151-7-16**] Dischar...,"['service', 'addendum', 'radiologic', 'studies...","['01193', '4254', '42731', '2639', '2762', '50...",RT LOWER LOBE PNEUMONIA
1,1,22532,167853.0,Discharge summary,Medicare,EMERGENCY,Medicare,Admission Date: [**2151-7-16**] Dischar...,"['history', 'present', 'illness', 'patient', '...","['01193', '4254', '42731', '2639', '2762', '50...",RT LOWER LOBE PNEUMONIA
32,32,13702,107527.0,Discharge summary,Medicare,EMERGENCY,Medicare,Admission Date: [**2118-6-2**] Discharg...,"['date', 'birth', 'sex', 'f', 'service', 'micu...","['5191', '49121', '51881', '486', '2761', '244...",CHRONIC OBSTRUCTIVE PULMONARY DISEASE
79,79,13702,167118.0,Discharge summary,Medicare,ELECTIVE,Medicare,Admission Date: [**2119-5-4**] D...,"['service', 'cardiothoracic', 'allergies', 'am...","['5191', '5185', '496', '2762', '45340', '5533']",TRACHEALBRONCHEAL MALACEA/SDA
186,186,13702,196489.0,Discharge summary,Medicare,EMERGENCY,Medicare,Admission Date: [**2124-7-21**] ...,"['service', 'medicine', 'allergies', 'amlodipi...","['51884', '5849', '34830', '49121', '2760', '4...",ASTHMA;COPD EXACERBATION


In [48]:
df_discharge.to_csv("../data/df_discharge.csv")

## Reduced dataset with only diabetic patients

Now we reduce the size of the dataset considering only the patients that are classified with a code related with diabetes (250.00)

In [3]:
reduced_df = Utils().extract_diabete_only(df_discharge)

NameError: name 'df_discharge' is not defined

In [57]:
reduced_df.shape

(9829, 11)

In [58]:
reduced_df.to_csv("../data/only_diabete.csv")

# Medical Abbreviations

In [18]:
abbreviations = []
files = ["../data/abbreviations/" + f for f in listdir("../data/abbreviations/") if isfile(join("../data/abbreviations/", f))]
files.remove('../data/abbreviations/.DS_Store')
print(files)
for filename in files:
    df = pd.read_csv(filename)
    abbreviations.append(df)

abbreviations = pd.concat(abbreviations, axis=0, ignore_index=True)

['../data/abbreviations/File 9.csv', '../data/abbreviations/File_8.csv', '../data/abbreviations/File 1.csv', '../data/abbreviations/File 0.csv', '../data/abbreviations/File 2.csv', '../data/abbreviations/File 3.csv', '../data/abbreviations/File 7.csv', '../data/abbreviations/File 6.csv', '../data/abbreviations/File 4.csv', '../data/abbreviations/File 5.csv']


In [19]:
abbreviations.shape

(3171, 3)

In [20]:
abbreviations.head()

,Abbreviation/Shorthand,Meaning,More Info
0,2D-RT,two-dimensional radiotherapy,NaN
1,2VD,two vessels disease,NaN
2,3D-CRT,three-dimensional conformal radiotherapy,NaN
3,3DCRT,three-dimensional conformal radiotherapy,NaN
4,3VD,three vessels diseases,NaN


In [21]:
abbreviations[abbreviations['Abbreviation/Shorthand'] == "VAC"]

,Abbreviation/Shorthand,Meaning,More Info
1353,VAC,vacuum-assisted closure,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...


In [22]:
abbreviations = abbreviations.drop_duplicates(subset=['Abbreviation/Shorthand'], keep=False)

In [23]:
abbreviations.shape

(2432, 3)

In [24]:
abbreviations_dict = {}

for _, row in abbreviations.iterrows():
    abbreviations_dict[row['Abbreviation/Shorthand']] = row['Meaning']

pickle.dump(abbreviations, open("../data/abbreviations/abbreviations_dict.pkl", "wb"))

In [25]:
abbr = pickle.load(open("../data/abbreviations/abbreviations_dict.pkl", "rb"))